In [9]:
import sys
sys.path.append('./code')
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np
import time
import os
import random
import argparse
from vgg_face import *
from vgg_readout import *
import matplotlib.pyplot as plt
from facegenDataset import *
from FAUDataset import *
plt.switch_backend('agg')

class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.main = nn.Sequential(
            nn.Linear(1, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )
        
    def forward(self, x):
        return self.main(x)

def get_model():
    model = Net()
    return model, optim.Adam(model.parameters(), lr=1e-3)

def sos(ypred, y):
    "Sum of squares loss"
    return torch.sum((ypred - y) ** 2)

def get_jacobian(net, x, noutputs):
    x.requires_grad = True
    y = net(x)
    grad_params = torch.autograd.grad(y, x, create_graph=True)
    return grad_params[0]

# just a straight line with slope 1 and intercept 0
data = [0., 0.25, 0.5, 0.75, 1.0] 
labels = [0., 0.25, 0.5, 0.75, 1.0] 

model, opt = get_model()
jacobian_losses = []
reg_losses = []
for i in range(100):

    ypred = model(torch.tensor(data).view(-1, 1))
    reg_loss = sos(ypred, torch.tensor(labels).view(-1, 1))
    reg_losses.append(float(reg_loss))
    
    jacobian_loss = 0
    for x in [0., 0.25, 0.5, 0.75, 1.0]:
        jacobian = get_jacobian(model, torch.tensor([[x]]), 1)
        temp = torch.norm(jacobian) ** 2
        jacobian_loss += temp
    jacobian_losses.append(float(jacobian_loss))
    
    total_loss = reg_loss + 1e-1 * jacobian_loss
    
    jacobian_loss.backward()
    opt.step()
    opt.zero_grad()